##Ingestion del archivo "person.json"

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StringType, StructField, IntegerType

In [0]:
name_schema = StructType(fields=[
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
persons_schema = StructType(fields=[
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
])

In [0]:
persons_df = spark.read \
    .schema(persons_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/person.json")

In [0]:
persons_df.printSchema()

### Paso 2 - Renombrar las columnas y añadir nuevas columnas 
1. "personId" renombrar a "person_id"
2. Agregar las columnas "ingestion_date" y "enviroment"
3. Agregar la columna "name" a partir la la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col, concat, current_timestamp, lit

In [0]:
persons_with_colums_df = add_ingestion_date(persons_df) \
    .withColumnRenamed("personId", "person_id") \
    .withColumn("enviroment", lit(v_enviroment)) \
    .withColumn("name", concat(col("personName.forename"), lit(" "), col("personName.surname"))) \
    .withColumn("file_date", lit(v_file_date))

In [0]:
display(persons_with_colums_df)

### Paso 3 - Eliminar las columnas no "requeridas"

In [0]:
persons_final_df = persons_with_colums_df.drop(col("personName"))

### Paso 4 - Escribir la salida en un formato "Parquet"

In [0]:
#overwrite_partition(persons_final_df, "movie_silver", "persons", "file_date")

In [0]:
merge_condition = 'tgt.person_id = src.person_id AND tgt.file_date = src.file_date'
merge_delta_lake(persons_final_df, "movie_silver", "persons", silver_folder_path, merge_condition, "file_date")

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.persons
GROUP BY file_date;

In [0]:
display(spark.read.format("delta").load(f"{silver_folder_path}/persons"))

In [0]:
dbutils.notebook.exit("Success")